In [1]:
import pandas as pd
import requests
import time
import os
import csv
from io import StringIO
import urllib3
from datetime import datetime, timedelta

#### **Annual Maximums**

In [3]:
# function to extract annual maximums
def get_annual_maximums(input_folder, output_csv):
    durations = {
        "30-min": 1,
        "1-hour": 2,
        "2-hour": 4,
        "3-hour": 6,
        "6-hour": 12,
        "12-hour": 24,
        "24-hour": 48,
        "48-hour": 96,
        "72-hour": 144
    }

    # Initialize the output DataFrame
    output_data = []

    # Process each file in the input folder
    for file in os.listdir(input_folder):
        if file.endswith(".csv"):
            # Extract metadata from the filename
            parts = file.split("_")
            station_id = parts[1]
            latitude = float(parts[2])
            longitude = float(parts[3].replace(".csv", ""))

            # Read the file into a DataFrame
            file_path = os.path.join(input_folder, file)
            df = pd.read_csv(file_path)

            # Convert the time column to datetime
            df["time"] = pd.to_datetime(df["time"], errors='coerce')

            # Ensure the data is sorted by time
            df = df.sort_values(by="time").reset_index(drop=True)

            # Extract unique years from 2000 to 2024
            df['year'] = df['time'].dt.year
            years = range(2000, 2025)

            # Calculate annual maximums for each year and duration
            for year in years:
                year_data = df[df['year'] == year]

                if year_data.empty:
                    continue

                annual_max = {
                    "ID": station_id,
                    "lat": latitude,
                    "lon": longitude,
                    "year": year
                }

                # Compute the maximums for each duration
                for duration, window in durations.items():
                    if window == 1:
                        # 30-min maximums directly from the data
                        annual_max[duration] = year_data['precipitation'].max()
                    else:
                        # Calculate rolling sums for longer durations
                        rolling_sums = year_data['precipitation'].rolling(window=window, min_periods=1).sum()
                        annual_max[duration] = rolling_sums.max()

                output_data.append(annual_max)

    # Create a DataFrame from the results
    output_df = pd.DataFrame(output_data)

    # Save the results to the output CSV file
    output_df.to_csv(output_csv, index=False)

# input and output paths
input_folder = "output_filesv7"
output_csv = "annual_max.csv"

# Run function
get_annual_maximums(input_folder, output_csv)

print(f"Annual maximums saved to {output_csv}.")


Annual maximums saved to annual_max.csv.


#### **All-time Maximums (2000 to 2024) - Jude Approach**

In [2]:
# Function to compute all time maximums from different storms (Jude's Idea)

def get_alltime_maximums(input_folder: str, output_file: str):
    # Define the durations and corresponding window sizes
    durations = {
        "30-min": 1,
        "1-hour": 2,
        "2-hour": 4,
        "3-hour": 6,
        "6-hour": 12,
        "12-hour": 24,
        "24-hour": 48,
        "48-hour": 96,
        "72-hour": 144
    }

    # Initialize the output data1
    output_rows = []

    # Process each file in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".csv") and file_name.startswith("ts_"):
            # Extract ID, lat, and lon from file name
            parts = file_name.split("_")
            station_id = int(parts[1])
            lat = float(parts[2])
            lon = float(parts[3].replace(".csv", ""))

            # Read the CSV file
            file_path = os.path.join(input_folder, file_name)
            data = pd.read_csv(file_path)

            # Ensure datetime column is parsed and precipitation is numeric
            data['time'] = pd.to_datetime(data['time'], errors='coerce')
            data['precipitation'] = pd.to_numeric(data['precipitation'], errors='coerce')
            data = data.dropna()

            # Filter data to include only rows from 2000 to 2024
            data = data[(data['time'].dt.year >= 2000) & (data['time'].dt.year <= 2024)]

            # Process data for each duration
            top_max_values = {duration: [] for duration in durations}
            for duration, window_size in durations.items():
                # Calculate rolling sums for the specified duration
                rolling_sum = data['precipitation'].rolling(window=window_size).sum()
                rolling_data = pd.DataFrame({
                    "time": data['time'][window_size - 1:].reset_index(drop=True),
                    "value": rolling_sum.dropna().reset_index(drop=True)
                })

                # Sort by value and exclude overlapping windows
                non_overlapping_max = []
                used_indices = set()
                for idx, row in rolling_data.nlargest(len(rolling_data), 'value').iterrows():
                    if all(abs(idx - used_idx) >= window_size for used_idx in used_indices):
                        non_overlapping_max.append((row['value'], row['time'].year))
                        used_indices.add(idx)
                        if len(non_overlapping_max) == 25:
                            break

                # Store the top 25 maximums for the current duration
                top_max_values[duration] = non_overlapping_max

            # Create a row for the output
            for i in range(25):
                output_row = {
                    "ID": station_id,
                    "lat": lat,
                    "lon": lon
                }

                # Add maximum values and corresponding years for each duration
                for duration in durations:
                    if i < len(top_max_values[duration]):
                        value, year = top_max_values[duration][i]
                        output_row[duration] = value
                        output_row[f"year_{duration}"] = year
                    else:
                        output_row[duration] = None
                        output_row[f"year_{duration}"] = None

                output_rows.append(output_row)

    # Convert output data to DataFrame and save to CSV
    output_df = pd.DataFrame(output_rows)
    output_df.to_csv(output_file, index=False)

# Input and output paths
input_folder = "output_filesv7"
output_csv = "alltime_max.csv"

# Run the function
get_alltime_maximums(input_folder, output_csv)
print(f"All-time maximums saved to {output_csv}.")


All-time maximums saved to alltime_max.csv.


#### **Partial Duration Maximums - (Trimmed None)**

In [3]:
# function to calculate extract partial duration maximums (based on upper and lower bound threshold of annual maximums)

def get_partial_duration_maximums(input_folder: str, output_file: str, summary_file: str):
    durations = {
        "30-min": 1,
        "1-hour": 2,
        "2-hour": 4,
        "3-hour": 6,
        "6-hour": 12,
        "12-hour": 24,
        "24-hour": 48,
        "48-hour": 96,
        "72-hour": 144
    }

    all_output_rows = []
    summary_rows = []

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".csv") and file_name.startswith("ts_"):
            # Extract metadata from filename
            parts = file_name.split("_")
            station_id = int(parts[1])
            lat = float(parts[2])
            lon = float(parts[3].replace(".csv", ""))

            # Read the CSV
            file_path = os.path.join(input_folder, file_name)
            df = pd.read_csv(file_path)

            # Clean data
            df['time'] = pd.to_datetime(df['time'], errors='coerce')
            df['precipitation'] = pd.to_numeric(df['precipitation'], errors='coerce')
            df = df.dropna().sort_values("time").reset_index(drop=True)

            # Restrict to 2000–2024
            df = df[(df['time'].dt.year >= 2000) & (df['time'].dt.year <= 2024)]
            if df.empty:
                continue

            # Step 1: Calculate annual bounds for each duration
            df['year'] = df['time'].dt.year
            annual_bounds = {}
            for duration, window in durations.items():
                max_per_year = []
                for year, group in df.groupby('year'):
                    if group.empty:
                        continue
                    if window == 1:
                        max_val = group['precipitation'].max()
                    else:
                        max_val = group['precipitation'].rolling(window=window, min_periods=1).sum().max()
                    if pd.notna(max_val):
                        max_per_year.append(max_val)
                if max_per_year:
                    annual_bounds[duration] = (min(max_per_year), max(max_per_year))
                else:
                    annual_bounds[duration] = (None, None)

            # Step 2: Collect non-overlapping maxima for each duration
            duration_maxima = {}
            for duration, window_size in durations.items():
                lower, upper = annual_bounds[duration]
                if lower is None or upper is None:
                    duration_maxima[duration] = []
                    continue

                rolling_sum = df['precipitation'].rolling(window=window_size).sum()
                rolling_data = pd.DataFrame({
                    "time": df['time'][window_size - 1:].reset_index(drop=True),
                    "value": rolling_sum.dropna().reset_index(drop=True)
                })

                # Keep only values within bounds
                rolling_data = rolling_data[(rolling_data['value'] >= lower) & (rolling_data['value'] <= upper)]

                # Enforce non-overlapping
                selected = []
                used_idx = set()
                for idx, row in rolling_data.nlargest(len(rolling_data), 'value').iterrows():
                    if all(abs(idx - u) >= window_size for u in used_idx):
                        selected.append((row['value'], row['time'].year))
                        used_idx.add(idx)

                # Sort descending by value (ensuring year stays aligned)
                duration_maxima[duration] = sorted(selected, key=lambda x: x[0], reverse=True)

            # Step 3: Align durations — keep only rows where all durations have values
            min_len = min(len(v) for v in duration_maxima.values())
            for i in range(min_len):  # only keep complete rows
                row = {"ID": station_id, "lat": lat, "lon": lon}
                for duration in durations:
                    val, yr = duration_maxima[duration][i]
                    row[duration] = val
                    row[f"year_{duration}"] = yr
                all_output_rows.append(row)

            # Step 4: Add summary counts (still store all counts before trimming)
            summary_row = {"ID": station_id}
            for duration in durations:
                summary_row[f"{duration}-count"] = len(duration_maxima[duration])
            summary_rows.append(summary_row)

    # Save detailed output (only complete rows kept)
    output_df = pd.DataFrame(all_output_rows)
    output_df.to_csv(output_file, index=False)

    # Save summary file
    summary_df = pd.DataFrame(summary_rows)
    summary_df.to_csv(summary_file, index=False)

    print(f"Partial duration maximums saved to {output_file}")
    print(f"Summary saved to {summary_file}")


# Run Script 
input_folder = "output_filesv7"
output_file = "alltime_partial_duration_max.csv"
summary_file = "alltime_partial_duration_max_summary.csv"

get_partial_duration_maximums(input_folder, output_file, summary_file)


Partial duration maximums saved to alltime_max_partial_duration.csv
Summary saved to alltime_max_partial_duration_summary.csv


#### **Partial Duration Maximums - (Complete)**

In [ ]:
def get_partial_duration_maximums(input_folder: str, output_file: str, summary_file: str):
    durations = {
        "30-min": 1,
        "1-hour": 2,
        "2-hour": 4,
        "3-hour": 6,
        "6-hour": 12,
        "12-hour": 24,
        "24-hour": 48,
        "48-hour": 96,
        "72-hour": 144
    }

    output_rows = []
    summary_rows = []

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".csv") and file_name.startswith("ts_"):
            # Extract metadata from filename
            parts = file_name.split("_")
            station_id = int(parts[1])
            lat = float(parts[2])
            lon = float(parts[3].replace(".csv", ""))

            # Read the CSV
            file_path = os.path.join(input_folder, file_name)
            df = pd.read_csv(file_path)

            # Clean data
            df['time'] = pd.to_datetime(df['time'], errors='coerce')
            df['precipitation'] = pd.to_numeric(df['precipitation'], errors='coerce')
            df = df.dropna().sort_values("time").reset_index(drop=True)

            # Restrict to 2000–2024
            df = df[(df['time'].dt.year >= 2000) & (df['time'].dt.year <= 2024)]
            if df.empty:
                continue

            # Step 1: Calculate annual bounds for each duration
            df['year'] = df['time'].dt.year
            annual_bounds = {}
            for duration, window in durations.items():
                max_per_year = []
                for year, group in df.groupby('year'):
                    if group.empty:
                        continue
                    if window == 1:
                        max_val = group['precipitation'].max()
                    else:
                        max_val = group['precipitation'].rolling(window=window, min_periods=1).sum().max()
                    if pd.notna(max_val):
                        max_per_year.append(max_val)
                if max_per_year:
                    annual_bounds[duration] = (min(max_per_year), max(max_per_year))
                else:
                    annual_bounds[duration] = (None, None)

            # Step 2: Collect non-overlapping maxima for each duration
            duration_maxima = {}
            for duration, window_size in durations.items():
                lower, upper = annual_bounds[duration]
                if lower is None or upper is None:
                    duration_maxima[duration] = []
                    continue

                rolling_sum = df['precipitation'].rolling(window=window_size).sum()
                rolling_data = pd.DataFrame({
                    "time": df['time'][window_size - 1:].reset_index(drop=True),
                    "value": rolling_sum.dropna().reset_index(drop=True)
                })

                # Keep only values within bounds
                rolling_data = rolling_data[(rolling_data['value'] >= lower) & (rolling_data['value'] <= upper)]

                # Enforce non-overlapping
                selected = []
                used_idx = set()
                for idx, row in rolling_data.nlargest(len(rolling_data), 'value').iterrows():
                    if all(abs(idx - u) >= window_size for u in used_idx):
                        selected.append((row['value'], row['time'].year))
                        used_idx.add(idx)

                duration_maxima[duration] = selected

            # Step 3: Create wide-format rows
            max_len = max(len(v) for v in duration_maxima.values())
            for i in range(max_len):
                row = {"ID": station_id, "lat": lat, "lon": lon}
                for duration in durations:
                    if i < len(duration_maxima[duration]):
                        val, yr = duration_maxima[duration][i]
                        row[duration] = val
                        row[f"year_{duration}"] = yr
                    else:
                        row[duration] = None
                        row[f"year_{duration}"] = None
                output_rows.append(row)

            # Step 4: Add counts for summary file
            summary_row = {"ID": station_id}
            for duration in durations:
                summary_row[f"{duration}-count"] = len(duration_maxima[duration])
            summary_rows.append(summary_row)

    # Save detailed output (wide format)
    output_df = pd.DataFrame(output_rows)
    output_df.to_csv(output_file, index=False)

    # Save summary file (with -count suffix)
    summary_df = pd.DataFrame(summary_rows)
    summary_df.to_csv(summary_file, index=False)

    print(f"Partial duration maximums saved to {output_file}")
    print(f"Summary saved to {summary_file}")


# Run Script 
input_folder = "output_filesv7"
output_file = "alltime_partial_duration_max_raw.csv"
summary_file = "alltime_partial_duration_max_raw_summary.csv"

get_partial_duration_maximums(input_folder, output_file, summary_file)


#### **Extras (Alltime Maximums - Old Approach)**

In [ ]:
# Function to extract all-time maximums within the date range 2001 to 2023
def get_alltime_maximums(input_folder: str, output_file: str):
    # Define the durations and corresponding window sizes
    durations = {
        "30-min": 1,
        "1-hour": 2,
        "2-hour": 4,
        "3-hour": 6,
        "6-hour": 12,
        "12-hour": 24,
        "24-hour": 48,
        "48-hour": 96,
        "72-hour": 144
    }

    # Initialize the output DataFrame
    output_rows = []

    # Process each file in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".csv") and file_name.startswith("ts_"):
            # Extract ID, lat, and lon from file name
            parts = file_name.split("_")
            station_id = int(parts[1])
            lat = float(parts[2])
            lon = float(parts[3].replace(".csv", ""))

            # Read the CSV file
            file_path = os.path.join(input_folder, file_name)
            data = pd.read_csv(file_path)
            
            # Ensure datetime column is parsed and precipitation is numeric
            data['time'] = pd.to_datetime(data['time'], errors='coerce')
            data['precipitationCal'] = pd.to_numeric(data['precipitationCal'], errors='coerce')
            data = data.dropna()

            # Filter data to only include rows from 2001 to 2023
            data = data[(data['time'].dt.year >= 2001) & (data['time'].dt.year <= 2023)]

            # Create a dictionary to store the top 23 values for each duration
            top_max_values = {duration: [] for duration in durations}

            # Process data for each duration
            for duration, window_size in durations.items():
                # Calculate rolling sums for the specified duration
                rolling_sum = data['precipitationCal'].rolling(window=window_size).sum()
                rolling_data = pd.DataFrame({
                    "time": data['time'],
                    "value": rolling_sum
                }).dropna()
                
                # Extract top 23 maximums
                top_max = rolling_data.nlargest(23, 'value')

                # Store the top 23 values for the current duration
                top_max_values[duration] = top_max

            # Create 23 rows, one for each highest value
            for i in range(23):
                output_row = {
                    "ID": station_id,
                    "lat": lat,
                    "lon": lon
                }

                # For each duration, get the i-th highest value and the corresponding year
                for duration in durations:
                    if i < len(top_max_values[duration]):
                        top_value = top_max_values[duration].iloc[i]
                        output_row[duration] = top_value['value']
                        output_row[f"year_{duration}"] = top_value['time'].year
                    else:
                        output_row[duration] = None
                        output_row[f"year_{duration}"] = None

                # Append the row to the output list
                output_rows.append(output_row)

    # Combine all rows into a single DataFrame
    output_df = pd.DataFrame(output_rows)

    # Write to output CSV
    output_df.to_csv(output_file, index=False)

# Input and output paths
imerg_folder = "output_files"
output_csv = "alltime_max_2001_2023.csv"

# Run Function
get_alltime_maximums(imerg_folder, output_csv)

print(f"All-time maximums (Y2001 to Y2023) saved to {output_csv}.")
